In [1]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
%store -r mc_data

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        track_classifier = load_model('./models/track_classifier.h5')

In [4]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event
14   hasT
15   isInUT
16   isInVelo
17   isLong
18   isLong_fromB
19   isLong_fromB_more5
20   isLong_more5


In [5]:
mc_hits = mc_data.values

In [6]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r max_y
%store -r min_y
%store -r events

In [7]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [8]:
event_threshold = 15840

In [61]:
hits_total = mc_hits[mc_hits[:,13] > event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1) and (hit[0] != 11) and (hit[0] != -11):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), [float(hit[5][i]), int(hit[4][i])])

Total hits with threshold 199144


KeyboardInterrupt: 

In [9]:
def predictHeight(x0, x3, x_stereo, stereo_layer, first_layer, last_layer):
    
    z_stereo = zLays[stereo_layer]
    
    if (stereo_layer % 2 != 0):
        angle = 5
    else:
        angle = -5
        
    if (float(x3 - x0)  == 0):
        m = 1
    else:
        m = float(zLays[last_layer] - zLays[first_layer]) / float(x3 - x0)
    n = zLays[last_layer] - (m * x3)
    
    x_pred = (z_stereo - n) / m 
    beta = (x_pred - x_stereo) / (math.radians(angle) * z_stereo)
    return math.tan(beta) * z_stereo

In [32]:
#
# Test the NN model by using events not feeded as train 
#

def trackReconstruction(hits_total, index_tree):

    # Variables used for calculating metrics
    total_found = 0
    total = 0
    mean_size = np.zeros(3)

    # Variable defining the maximum number of hits gathered from the R-trees when using the predicted value from the NN
    max_hits_gathered = 12

    # Range in which valid hits are look up in the first R-tree
    long_range = 300

    X_classifier = []
    Y_classifier = []

    # For each particle of a given event
    for particle in hits_total:

        # Sometimes there are no PrHits for a given particle, so skip these ones
        if (len(particle[5]) > 1) and (particle[0] != 11) and (particle[0] != -11):

            found_num = 0            
            X_final = {}
            Zone_final = {}
            actual = -1

            # Extract all PrHits
            for i in range(0, len(particle[5])):
                if (actual != particle[8][i]):
                    X_final[particle[8][i]] = []
                    X_final[particle[8][i]].append(particle[5][i])
                    Zone_final[particle[8][i]] = particle[12][i]
                    actual = particle[8][i]
                    found_num += 1
                else:
                    X_final[actual].append(particle[5][i])

            # Check if there are minimum 12 hits found, as for the test, I am only working with complete tracks
            if not 0 in X_final or not 4 in X_final or not 8 in X_final:
                continue

            # List used to iterate over valid height found later on
            valid_y = {}

            # For each station
            for station in range(0,3):

                valid_y[station] = []

                # Compute the layer number based on station
                x0_layer = 0
                if (station == 1):
                    x0_layer = 4
                elif station == 2:
                    x0_layer = 8

                u_layer = x0_layer + 1
                v_layer = x0_layer + 2
                x1_layer = x0_layer + 3

                # Select both the first X position and UP/DOWN based on this
                first_hit = X_final[x0_layer][0]
                zone = Zone_final[x0_layer]

                # Predict all 3 following points based on the first hit selected
                predicted = (xlayer_model.predict(np.array([first_hit, x0_layer]).reshape(1,-1)))[0]
                mean_predicted = ((first_hit + predicted[0] + predicted[1] + predicted[2]) / 4)

                # Recolect all nearest hits from each layer using R-trees based on the X-position of the mean predicted
                found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, x1_layer, x1_layer, zone, zone), objects='raw'))
                found_layer3 = sorted(found, key=lambda x: np.abs(x[0]- mean_predicted))[:max_hits_gathered]
                arr_layer3 = np.unique(np.array(found_layer3), axis=0)

                found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, v_layer, v_layer, zone, zone), objects='raw'))
                found_layer2 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:max_hits_gathered]
                arr_layer2 = np.unique(np.array(found_layer2), axis=0)

                found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, u_layer, u_layer, zone, zone), objects='raw'))
                found_layer1 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:max_hits_gathered]
                arr_layer1 = np.unique(np.array(found_layer1), axis=0)

                for h3 in arr_layer3:

                    for h2 in arr_layer2:

                        # Predict height for each hit selected in the V layer
                        pred_h2 = predictHeight(first_hit, h3[0], h2[0], v_layer, x0_layer, x1_layer)

                        if (pred_h2 > 0) and (pred_h2 > max_y) or (pred_h2 < 0) and (pred_h2 < min_y):
                            continue

                        # And for each hit in the V layer, we look up at hits in the U layer with the same height
                        for h1 in arr_layer1:

                            # Predict height for hit in the U layer
                            pred_h1 = predictHeight(first_hit, h3[0], h1[0], u_layer, x0_layer, x1_layer)
                            if (pred_h1 > 0) and (pred_h1 > max_y) or (pred_h1 < 0) and (pred_h1 < min_y):
                                continue

                            if np.abs(pred_h2 - pred_h1) < 15.5:
                                # Insert into a new R-tree to enable search by height
                                # Store the valid height for later iterate over it
                                valid_y[station].append([first_hit, h1[0], h2[0], h3[0], pred_h2, (h1[1],h2[1],h3[1])])

            valid_tracks = []
            for val in valid_y[0]:
                if (len(valid_y[1]) > 0 and len(valid_y[2]) > 0):
                    second_sort = sorted(valid_y[1], key = lambda p: np.abs(p[4] - val[4]))
                    if (np.abs(second_sort[0][4] - val[4]) < 100):
                        third_sort = sorted(valid_y[2], key = lambda p: np.abs(p[4] - second_sort[0][4]))
                        if np.abs(third_sort[0][4] - second_sort[0][4]) < 100:
                            valid_tracks.append([val, second_sort[0], third_sort[0]])

            
            repeated = 0
            len_tracks = len(valid_tracks)
            for track in valid_tracks:
                found = 0
                for station in range(0,3):
                    x0_layer = 0
                    if (station == 1):
                        x0_layer = 4
                    elif station == 2:
                        x0_layer = 8

                    u_layer = x0_layer + 1
                    v_layer = x0_layer + 2
                    x1_layer = x0_layer + 3

                    pos = 0
                    zone = 0
                    if (track[station][4] > 0):
                        zone = 1

                    for id in track[station][5]:
                        if (id in particle[4]):
                            found += 1
                            #index_tree.delete(int(id), (track[station][pos], track[station][pos], x0_layer + pos + 1, x0_layer + pos + 1, zone, zone))
                        pos += 1

                if (found > 5):
                    X_classifier.append(track[0][:5] + track[1][:5] + track[2][:5])
                    Y_classifier.append(1)
                    len_tracks -= 1
                    repeated += 1
                else:
                    X_classifier.append(track[0][:5] + track[1][:5] + track[2][:5])
                    Y_classifier.append(0)

            if (repeated > 0):
                total_found += 1

            mean_size[0] += (len_tracks)
            total += 1
    #print("\nMean Total Found: %f" % (total_found/total))
    #print("\nMean Size: %f" % (mean_size[0]/total))
    return X_classifier, Y_classifier, total_found/total, mean_size[0]/total
    

In [36]:
mean = size = done = 0
test_events = events[3:]

X_classifier = []
Y_classifier = []

p = index.Property()
p.dimension = 3
    
for event in test_events:
    particles_test_event = mc_hits[mc_hits[:,13] == event]
    # print("Event length %d " % len(particles_test_event))
    idx = {}
    mean_val = size_val = 0
    index_tree = index.Index(properties=p, interleaved=False)
    for hit in particles_test_event:
        if (len(hit[5]) > 1) and (hit[0] != 11) and (hit[0] != -11):
            for i in range(0,len(hit[5])):
                index_tree.insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), [float(hit[5][i]), int(hit[4][i])])

    X, Y, mean_val, size_val = trackReconstruction(particles_test_event, index_tree)
    
    X_classifier += X
    Y_classifier += Y
    mean += mean_val
    size += size_val
    done += 1
    print("Event %d/%d \t Acc: %f Size %f Proportion %f" % (done,len(test_events), mean_val, size_val, Y.count(0)/(Y.count(1) + 1)))
    
print("\nFinal Result:")
print("\t X_Size : %f" % (len(X_classifier)))
print("\t Y_Size: %f" % (len(Y_classifier)))
print("\t Y count 0 : %f" % (Y_classifier.count(0)))
print("\t Y count 1: %f" % (Y_classifier.count(1)))
print("\t Mean: %f" % (mean/len(test_events)))
print("\t Size: %f" % (size/len(test_events)))

Event 1/926 	 Acc: 0.526882 Size 0.591398 Proportion 0.687500
Event 2/926 	 Acc: 0.652439 Size 2.152439 Proportion 1.440816
Event 3/926 	 Acc: 0.678571 Size 0.285714 Proportion 0.266667
Event 4/926 	 Acc: 0.568627 Size 0.411765 Proportion 0.420000
Event 5/926 	 Acc: 0.645161 Size 1.072581 Proportion 0.904762
Event 6/926 	 Acc: 0.688312 Size 0.350649 Proportion 0.216000
Event 7/926 	 Acc: 0.658228 Size 1.683544 Proportion 1.254717
Event 8/926 	 Acc: 0.583333 Size 0.000000 Proportion 0.000000
Event 9/926 	 Acc: 0.532258 Size 0.080645 Proportion 0.104167
Event 10/926 	 Acc: 0.531915 Size 0.063830 Proportion 0.090909
Event 11/926 	 Acc: 0.628571 Size 0.000000 Proportion 0.000000
Event 12/926 	 Acc: 0.696970 Size 0.242424 Proportion 0.210526
Event 13/926 	 Acc: 0.733333 Size 0.083333 Proportion 0.068493
Event 14/926 	 Acc: 0.666667 Size 0.166667 Proportion 0.144231
Event 15/926 	 Acc: 0.609626 Size 3.802139 Proportion 2.585455
Event 16/926 	 Acc: 0.465116 Size 0.000000 Proportion 0.000000
E

Event 131/926 	 Acc: 0.647059 Size 0.078431 Proportion 0.081633
Event 132/926 	 Acc: 0.833333 Size 0.083333 Proportion 0.083333
Event 133/926 	 Acc: 0.695652 Size 0.021739 Proportion 0.022222
Event 134/926 	 Acc: 0.739130 Size 0.347826 Proportion 0.238806
Event 135/926 	 Acc: 0.588235 Size 0.098039 Proportion 0.113636
Event 136/926 	 Acc: 0.526667 Size 6.960000 Proportion 4.226721
Event 137/926 	 Acc: 0.589474 Size 1.610526 Proportion 1.204724
Event 138/926 	 Acc: 0.559524 Size 2.785714 Proportion 1.933884
Event 139/926 	 Acc: 0.606383 Size 0.542553 Proportion 0.472222
Event 140/926 	 Acc: 0.610526 Size 4.936842 Proportion 2.977778
Event 141/926 	 Acc: 0.647059 Size 1.421569 Proportion 0.960265
Event 142/926 	 Acc: 0.676471 Size 0.308824 Proportion 0.250000
Event 143/926 	 Acc: 0.686747 Size 0.650602 Proportion 0.556701
Event 144/926 	 Acc: 0.621951 Size 0.878049 Proportion 0.679245
Event 145/926 	 Acc: 0.597015 Size 2.022388 Proportion 1.557471
Event 146/926 	 Acc: 0.662338 Size 0.116

Event 260/926 	 Acc: 0.666667 Size 1.041667 Proportion 0.796178
Event 261/926 	 Acc: 0.645390 Size 1.787234 Proportion 1.050000
Event 262/926 	 Acc: 0.584906 Size 0.169811 Proportion 0.180000
Event 263/926 	 Acc: 0.559322 Size 4.265537 Proportion 2.467320
Event 264/926 	 Acc: 0.717391 Size 0.000000 Proportion 0.000000
Event 265/926 	 Acc: 0.651685 Size 0.651685 Proportion 0.500000
Event 266/926 	 Acc: 0.575610 Size 3.639024 Proportion 2.486667
Event 267/926 	 Acc: 0.588235 Size 0.731092 Proportion 0.617021
Event 268/926 	 Acc: 0.564767 Size 4.471503 Proportion 2.876667
Event 269/926 	 Acc: 0.600000 Size 1.742857 Proportion 1.574194
Event 270/926 	 Acc: 0.611842 Size 3.828947 Proportion 2.318725
Event 271/926 	 Acc: 0.612903 Size 0.387097 Proportion 0.369231
Event 272/926 	 Acc: 0.565517 Size 3.103448 Proportion 2.556818
Event 273/926 	 Acc: 0.694444 Size 0.166667 Proportion 0.129032
Event 274/926 	 Acc: 0.663158 Size 0.610526 Proportion 0.429630
Event 275/926 	 Acc: 0.550000 Size 0.000

Event 389/926 	 Acc: 0.685714 Size 0.000000 Proportion 0.000000
Event 390/926 	 Acc: 0.636364 Size 0.036364 Proportion 0.034483
Event 391/926 	 Acc: 0.587500 Size 2.575000 Proportion 1.775862
Event 392/926 	 Acc: 0.705882 Size 0.000000 Proportion 0.000000
Event 393/926 	 Acc: 0.682927 Size 0.048780 Proportion 0.039216
Event 394/926 	 Acc: 0.513889 Size 4.337963 Proportion 3.703557
Event 395/926 	 Acc: 0.585366 Size 0.353659 Proportion 0.315217
Event 396/926 	 Acc: 0.556962 Size 1.000000 Proportion 0.918605
Event 397/926 	 Acc: 0.605769 Size 0.817308 Proportion 0.648855
Event 398/926 	 Acc: 0.593103 Size 3.572414 Proportion 2.712042
Event 399/926 	 Acc: 0.736842 Size 0.000000 Proportion 0.000000
Event 400/926 	 Acc: 0.471264 Size 1.034483 Proportion 1.250000
Event 401/926 	 Acc: 0.627451 Size 0.039216 Proportion 0.033898
Event 402/926 	 Acc: 0.589286 Size 2.297619 Proportion 1.608333
Event 403/926 	 Acc: 0.604651 Size 0.488372 Proportion 0.375000
Event 404/926 	 Acc: 0.597403 Size 0.103

KeyboardInterrupt: 

In [44]:
len(X_classifier[0])

15

In [20]:
X_classifier = []
Y_classifier = []

total_found = 0
total = 0
long_range = 300

for particle in hits_total:
    
    if (len(particle[5]) > 1) and (particle[0] != 11) and (particle[0] != -11):
        
        found_num = 0
        found_layers = np.zeros(6)
        # Extract only 12 hits, as some of them are really close
        X_final = {}
        Zone_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
                Zone_final[particle[8][i]] = particle[12][i]
                found_num += 1
            else:
                X_final[actual].append(particle[5][i])
                
        if (found_num < 12):
            continue
        
        found_num = 0
        tmp_found = 0
        for station in range(0,3):
            
            # Compute the layer number based on station
            x0_layer = 0
            if (station == 1):
                x0_layer = 4
            elif station == 2:
                x0_layer = 8

            u_layer = x0_layer + 1
            v_layer = x0_layer + 2
            x1_layer = x0_layer + 3

            # Select both the first X position and UP/DOWN based on this
            first_hit = X_final[x0_layer][0]
            zone = Zone_final[x0_layer]
            
            found_layers[0] = first_hit

            predicted = (xlayer_model.predict(np.array([first_hit, x0_layer]).reshape(1,-1)))[0]
            mean_predicted = ((first_hit + predicted[0] + predicted[1] + predicted[2]) / 4)

            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, x1_layer, x1_layer, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer3)):
                if (found_layer3[i] in X_final[x1_layer]):
                    found_num += 1
                    found_layers[3] = found_layer3[i]
                    break

            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, v_layer, v_layer, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer2)):
                if (found_layer2[i] in X_final[v_layer]):
                    found_num += 1
                    found_layers[2] = found_layer2[i]
                    break

            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, u_layer, u_layer,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer1)):
                if (found_layer1[i] in X_final[u_layer]):
                    found_num += 1
                    found_layers[1] = found_layer1[i]
                    break

            if (found_num == 3):
                
                u_hit = random.choice(found_layer1)
                v_hit = random.choice(found_layer2)
                retries_x1 = 0
                while x1_hit == first_hit and retries_x1 != 4:
                    x1_hit = random.choice(found_layer3)
                    retries_x1 += 1
                
                if (first_hit == found_layers[3]):
                    continue
                
                found_layers[4] = predictHeight(first_hit, found_layers[3], found_layers[1], u_layer, x0_layer, x1_layer)
                found_layers[5] = predictHeight(first_hit, found_layers[3], found_layers[2], v_layer, x0_layer, x1_layer)

                if (retries_x1 < 4):
                    fake_u_height = predictHeight(first_hit, x1_hit, u_hit, u_layer, x0_layer, x1_layer)
                    fake_v_height = predictHeight(first_hit, x1_hit, v_hit, v_layer, x0_layer, x1_layer)
                    fake_track = np.array([first_hit, u_hit,v_hit,x1_hit, fake_u_height, fake_v_height])

                    true_track = found_layers
                    retries = 0

                    while (fake_track == true_track).all() and retries != 4:
                        u_hit = random.choice(found_layer1)
                        v_hit = random.choice(found_layer2)
                        retries_x1 = 0
                        while x1_hit == first_hit and retries_x1 != 4:
                            x1_hit = random.choice(found_layer3)
                            retries_x1 += 1
                        if (retries_x1 == 4):
                            retries = 4
                            break
                        fake_u_height = predictHeight(first_hit, x1_hit, u_hit, u_layer, x0_layer, x1_layer)
                        fake_v_height = predictHeight(first_hit, x1_hit, v_hit, v_layer, x0_layer, x1_layer)
                        fake_track = np.array([first_hit, u_hit, v_hit, x1_hit, fake_u_height, fake_v_height])
                        retries += 1

                    if (retries == 4):
                        X_classifier.append(true_track)
                        Y_classifier.append(1)
                    else:
                        X_classifier.append(true_track)
                        Y_classifier.append(1)
                        X_classifier.append(fake_track)
                        Y_classifier.append(0)
                else:
                    X_classifier.append(true_track)
                    Y_classifier.append(1)
                
                tmp_found += 1
                    
                
        if (tmp_found > 2):
            total_found += 1
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

NameError: name 'hits_total' is not defined

In [46]:
track_classifier = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
track_classifier.add(Dense(24, activation='relu', input_dim=15))
track_classifier.add(Dense(8, activation='relu'))
track_classifier.add(Dense(1, activation='sigmoid'))
track_classifier.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 113109
X Test Length 37704
Y Train length: 113109
Y Test length: 37704


In [50]:
track_classifier.fit(X_train, y_train, epochs=15, batch_size=20, verbose=1, validation_split=0.1)

Train on 101798 samples, validate on 11311 samples
Epoch 1/15
101798/101798 [==============================] - 7s 73us/sample - loss: 0.2816 - acc: 0.9034 - val_loss: 0.2582 - val_acc: 0.9196
Epoch 2/15
101798/101798 [==============================] - 8s 81us/sample - loss: 0.2770 - acc: 0.9067 - val_loss: 0.2523 - val_acc: 0.9151
Epoch 3/15
101798/101798 [==============================] - 9s 84us/sample - loss: 0.2746 - acc: 0.9074 - val_loss: 0.2538 - val_acc: 0.9152
Epoch 4/15
101798/101798 [==============================] - 8s 77us/sample - loss: 0.2741 - acc: 0.9073 - val_loss: 0.2665 - val_acc: 0.9127
Epoch 5/15
101798/101798 [==============================] - 9s 90us/sample - loss: 0.2710 - acc: 0.9083 - val_loss: 0.2467 - val_acc: 0.9184
Epoch 6/15
101798/101798 [==============================] - 9s 90us/sample - loss: 0.2691 - acc: 0.9089 - val_loss: 0.2495 - val_acc: 0.9175
Epoch 7/15
101798/101798 [==============================] - 8s 81us/sample - loss: 0.2685 - acc: 0.9084

In [53]:
score, acc = track_classifier.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

37704/37704 [==============================] - 1s 32us/sample - loss: 0.2346 - acc: 0.9155
Test score: 0.2345634160314167
Test accuracy: 0.9155262


In [54]:
track_classifier.save("./models/track_classifier_all_stations_v2.h5")

In [48]:
import datetime as dt
X_test = []

total_found = 0
total = 0
long_range = 300

hits_total = mc_hits[mc_hits[:,13] < event_threshold]

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        print(predicted)
        
        if (4 in X_final):
            found = list(idx[particle[13]].intersection((predicted[3] - long_range, predicted[3] + long_range, 4, 4,zone,zone),objects='raw'))
            found_layer4 = sorted(found, key=lambda x: np.abs(x - predicted[3]))[:10]
            found_bool += 1
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10]
            found_bool += 1
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10]
            found_bool += 1
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10]
            found_bool += 1
            
        if (found_bool == 4):
            retries = 0
            
            valid_list = []
            max_val = 0
            for h4 in found_layer4:
                for h3 in found_layer3:
                    for h2 in found_layer2:
                        for h1 in found_layer1:
                            pred = (track_classifier.predict(np.array([X_final[0][0], h1, h2, h3, h4]).reshape(1,-1))[0])[0]
                            if pred > max_val:
                                valid = [X_final[0][0], h1, h2, h3, h4]
                                max_val = pred
            
            if valid[1] in X_final[1] and valid[2] in X_final[2] and valid[3] in X_final[3] and valid[4] in X_final[4]:
                total_found += 1
                
            print(total/354)
            print(total_found/354)
            #print(total_found/354)
            #print(total_val)
            #print('\n')
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

[1344.3711 1331.2653 1353.7424 1449.1339]
0.0
0.0
[261.86023 264.366   268.9563  289.13846]
0.002824858757062147
0.0
[-244.05386 -283.24338 -305.46356 -349.80124]
0.005649717514124294
0.0
[139.9398  144.20387 146.77966 158.49117]
0.00847457627118644
0.0
[1401.4517 1387.5227 1410.9429 1510.3002]


KeyboardInterrupt: 